In [1]:
import cv2
import numpy as np
import tensorflow as tf
from IPython.display import display, HTML
from base64 import b64encode


In [ ]:
#yha pr train model load karenge
try:
    model = tf.keras.models.load_model('deepfake_detector_resnet_lstm.keras')  
    print("✅ Model loaded successfully.")
    model.summary() 
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise


In [ ]:
IMG_SIZE = (224, 224)  
FRAME_COUNT = 30       


In [ ]:
def preprocess_frame(frame):
    """Preprocesses a single frame to match ResNet50 training."""
    frame = cv2.resize(frame, IMG_SIZE)
    frame = frame.astype(np.float32)
    frame = tf.keras.applications.resnet50.preprocess_input(frame)
    return frame


In [ ]:
def extract_frames(video_path):
    """Extracts a fixed sequence of frames from the video."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"❌ Could not open video at {video_path}.")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // FRAME_COUNT)
    print(f"📽️ Total frames in video: {total_frames}, Frame interval: {frame_interval}")

    frames = []
    for i in range(0, total_frames, frame_interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            print(f"⚠️ Failed to read frame at index {i}")
            break
        processed_frame = preprocess_frame(frame)
        frames.append(processed_frame)

    cap.release()
    print(f"📸 Extracted {len(frames)} frames before padding.")

    # Pad with blank frames if needed
    blank_frame = np.zeros((IMG_SIZE[0], IMG_SIZE[1], 3), dtype=np.float32)
    blank_frame = tf.keras.applications.resnet50.preprocess_input(blank_frame)
    while len(frames) < FRAME_COUNT:
        frames.append(blank_frame)

    frames = np.array(frames[:FRAME_COUNT])
    print(f"✅ Final frames shape: {frames.shape}")
    return frames


In [ ]:
def classify_video(video_path, model):
    """Classifies a video as real or fake using frame sequences."""
    frames = extract_frames(video_path)
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    print(f"📊 Input shape to model: {frames.shape}")

    prediction = model.predict(frames, verbose=0)[0][0]
    predicted_label = "Fake" if prediction > 0.15 else "Real"

    return predicted_label, prediction


In [ ]:
def display_video(video_path):
    """Displays a video in a Jupyter Notebook."""
    try:
        with open(video_path, 'rb') as f:
            mp4 = f.read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        return HTML(f"""
        <video width=400 controls>
              <source src="{data_url}" type="video/mp4">
        </video>
        """)
    except FileNotFoundError:
        print(f"❌ Error: Video file not found at {video_path}.")
        return None
    except Exception as e:
        print(f"❌ Unexpected error occurred: {e}")
        return None


In [ ]:
video_file = "videos/v7.mp4"  # Change this path to your test video

try:
    # Display the video in Jupyter Notebook
    video_display = display_video(video_file)
    if video_display is not None:
        display(video_display)

    #for classification purpose
    label, probability = classify_video(video_file, model)
    print(f"🎯 The video is classified as: {label}")
    print(f"📊 Probability of being fake: {probability:.4f}")

except ValueError as e:
    print(f"❌ Error during video processing: {e}")
except Exception as e:
    print(f"❌ Unexpected error occurred: {e}")


📽️ Total frames in video: 94, Frame interval: 3
📸 Extracted 32 frames before padding.
✅ Final frames shape: (30, 224, 224, 3)
📊 Input shape to model: (1, 30, 224, 224, 3)
🎯 The video is classified as: Fake
📊 Probability of being fake: 0.1937
